# Appendix B: Problem 13

Estimation of OIIP and GIIP from scenario described by problems 11 and 12
Assuming:
 - Constant temperature
 - Constant oil composition
 - Constant gas composition
 - Initial CGR = 40 stb/mmscf
 - Surface condensate gravity = 60 deg API (gamma_og = 0.739)
 - Surface gas gravity for reservoir gas slightly less than surface gas gravity for reservoir oil (gamma_gg = 0.80)

In [46]:
# Given
import math
gor = 900 # scf/stb
sg_gas_total = 0.85
api_oil = 36 # deg API
p1 = 3300 #psia
t = 190 # deg F
h1 = 7200 # ft ss
top_structure = 6000 # ft ss
gas_cap_depth = 6500 # ft ss
woc = 7300 # ft ss
HCPV = 40e6 # cuft/ft
Pbub = 3090 # psia
R = 10.73146 # Universal gas constant, psia-ft3/R-lbm mol
dpdh = 0.2977 # Oil leg pressure gardient calculated in problem 12 (psi/ft)
rho_ob = 45.29384474245426 # lbm/cuft, calculated in problem 12 (rho_po)

#Assumed
cgr = 40 # stb/mmscf
gamma_og = 0.739
gamma_gg = 0.80
cond_api = 60

In [47]:
# Gas and oil leg HCPV's
Vphcg = HCPV*(gas_cap_depth - top_structure) # reservoir cf
Vphco = HCPV*(woc - gas_cap_depth)/5.614583 # reservoir bbls

# Condensate MW estimation
Mo = 6084/(cond_api - 5.9)

# Gas cap FWS gravity
gamma_fws = (gamma_gg+4580*cgr*1e-6*gamma_og)/(1+133000*cgr*1e-6*(gamma_og/Mo)) # Eq 3.55

Tpchc = 187 + 330 * gamma_fws - 71.5 * gamma_fws**2 # Eq 3.49a
Ppchc = 706 - 51.7 * gamma_fws - 11.1 * gamma_fws**2 # Eq 3.49b

# At GOC, P = Pbub = 3,090psia (calculated in Problem 12), so reduced properties are;
Tpr_goc = (t+460)/Tpchc
Ppr_goc = Pbub/Ppchc

In [48]:
# Copying Hall-Yarborough Z factor solution from Problem 6
from scipy.optimize import fsolve
import math
def fy(y, *args):
    alpha, Pr, t = args
    return -alpha * Pr + (y + y ** 2 + y ** 3 - y ** 4) / (1 - y) ** 3 - (14.76 * t - 9.76 * t ** 2 + 4.58 * t ** 3) * y ** 2 + (90.7 * t - 242.2 * t ** 2 + 42.4 * t ** 3) * y ** (2.18 + 2.82 * t)
    
def Zfac(Tr, Pr):
    t = 1 / Tr
    alpha = 0.06125 * t * math.exp(-1.2 * (1 - t) ** 2)
    args = (alpha, Pr, t)
    y = 0.001
    y = fsolve(fy, y, args=args)
    return alpha * Pr / y

In [49]:
# Z-factor at GOC is
Z_goc = Zfac(Tpr_goc, Ppr_goc)[0]

# Gas density at GOC;
Mg = gamma_fws * 28.97 # Average MW of gas cap = SG * Air MW
rhog_goc = (Pbub * Mg)/(Z_goc * R * (t + 460))

# Gas pressure gradient
dpdh_g = rhog_goc/144 #(psi/ft)

# Average gas cap pressure is then taken as half way up the gas leg
Prg = Pbub - dpdh_g*(gas_cap_depth - top_structure)/2

# Pseudo reduced pressure, Z-factor and 1/Bg at this average gas pressure
Ppr = Prg / Ppchc
Z = Zfac(Tpr_goc, Ppr)[0]
Bg_wet = (Z * (t + 460) / (Prg * 35.37))
one_on_Bg_wet = 1 / Bg_wet

# Need to calculate the dry gas FVF
Cog = 133000*gamma_og/Mo # Conversion from stock-tank condensate to equivalent surface gas scf/stb Eq 7.13 (Note: misprint in example incorrectly multiplies the MW when it should divide)
Bgd = Bg_wet*(1+Cog*cgr*1e-6) # Dry gas expansion factor Eq 7.12
one_on_Bg_dry = 1/Bgd

# Next, average oil leg pressure and oil FVF at this pressure using Standing Correlation
Pro = Pbub + dpdh * (woc - gas_cap_depth)/2
sg_oil = 141.5/(131.5+ api_oil)
A = gor * (sg_gas_total / sg_oil)**0.5 + 1.25*t
Bob = 0.9759 + 12e-5*A**1.2 # Bubble point oil FVF, Eq 3.111

# Next estimate understaurated oil FVF using an estimate of oil compressibility
A = 1e-5 * (5*gor + 17.2 * t - 1180 * sg_gas_total + 12.61 * api_oil - 1433)
Co = A/Pro # psi^-1 Eq 3.107
Bo = Bob * math.exp(Co*(Pbub - Pro)) # Eq 3.105

# Alternative 'more exact' approach offered
Co_alt = 1e-6*math.exp((rho_ob + 0.004347*(Pro - Pbub)-79.1)/(1.141e-4*(Pro - Pbub)-12.938)) # Eq 3.109
A_alt = Co_alt*Pro             # Eq 3.107 - **Note, I believe the worked example here is incorrect, as it doesnt use the 'more exact' Co
Bo_alt = Bob*(Pbub/Pro)**A_alt #Eq 3.108

# Initial OIP in oil column
No = Vphco / Bo_alt # stb

# Initial (dry) gas GIP
Gg = Vphcg * one_on_Bg_dry  # GIIP (cuft), **Note example incorrectly has Vphco instead of Vphcg

# Initial condensate in place in gas column
Ng = Gg * cgr * 1e-6 # stb condensate

# Initial gas in solution in oil column
Go = No * gor # scf solution gas

# Initial stock tank oil + condensate
N = No + Ng

# Initial Free gas + solution gas
G = Gg + Go

print('                      Oil leg Avg Bo:', round(Bo_alt,4), 'rb/stb')
print('                    Gas Leg Avg 1/Bg:', round(one_on_Bg_dry,1), 'dry scf/wet rcf')
print('     Total oil + Condensate in place:', round(No/1e6,1), 'MM stb')
print('Total free + associated gas in place:', round(G/1e12,3), 'TCF')

                      Oil leg Avg Bo: 1.5327 rb/stb
                    Gas Leg Avg 1/Bg: 200.2 dry scf/wet rcf
     Total oil + Condensate in place: 3718.6 MM stb
Total free + associated gas in place: 7.351 TCF


In [50]:
Z

0.804805847637417